# Visualizing Data in Roman ASDF Files

***

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Research Nexus" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Introduction
In this tutorial, we demonstrate how to visualize and explore Roman WFI image data arrays. We separate this tutorial into two high-level components:
1. creating static images with world coordinate system (WCS) overlays; and
2. using the Jdaviz tool to interactively explore WFI image data.

We focus on how to visualize WFI Level 2 (L2; calibrated rate image) data in ASDF format. For WFI, L2 indicates that the data have been processed to flag and/or correct for detector-level effects (e.g., saturation, classic non-linearity, etc.) and the resultants fitted into a count rate image. Each L2 ASDF file contains a single WFI detector, thus a complete WFI exposure is made up of 18 L2 files. For more information WFI L2 files, please see the RDox article on [Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products#DataLevelsandProducts-Level2-CalibratedExposuresLevel2).

## Imports
- *astropy.visualization.simple_norm* for automatically scaling image arrays
- *astropy.coordinates.SkyCoord* to create Python objects containing sky coordinate transforms
- *astropy.table.Table* to create Astropy tables
- *astropy.wcs.WCS* to create Astropy WCS objects
- *copy* to copy Python objects in memory
- *matplotlib.pyplot* for creating static image previews
- *numpy* for array calculations and manipulation
- *jdaviz.Imviz* to examine Wide Field Instrument (WFI) images interactively
- *roman_datamodels* for opening Roman WFI ASDF files
- *asdf* for opening Roman WFI ASDF files
- *time* for creating pauses in the notebook cells
- *s3fs* to access data in an AWS S3 bucket

In [ ]:
%matplotlib inline
from astropy.visualization import simple_norm
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.wcs import WCS
import copy
import matplotlib.pyplot as plt
import numpy as np

from jdaviz import Imviz
import roman_datamodels as rdm
import asdf
import s3fs
import time

## Loading data
We use the same data file in both examples. Note that the static image portions can be used on any `numpy.ndarray` object, and the WCS axes may be optionally included with any `astropy.wcs.WCS` object. Also, while the Jdaviz example is designed to work on Roman data, Jdaviz can also work with other data and file types. A link to the Jdaviz documentation is provided in the Additional Resources section below.

A complete explanation on how to load and work with Roman ASDF files is provided in the notebook tutorial [Working with ASDF](../working_with_asdf/working_with_asdf.ipynb). We read in the data using the `roman_datamodels` package:

In [ ]:
asdf_dir_uri = 's3://stpubdata/roman/nexus/soc_simulations/tutorial_data/'
fs = s3fs.S3FileSystem(anon=True)

asdf_file_uri = asdf_dir_uri + 'r0003201001001001004_0001_wfi01_f106_cal.asdf'
file = rdm.open(fs.open(asdf_file_uri, 'rb'))

## Static Image Display

### Plot an Image with Dynamic Scaling

Here we show how to use matplotlib and Astropy to plot the data array from our WFI image and scale the array automatically.

In [ ]:
# Set the image normalization. Here we use a inverse hyperbolic sine scale 
# with the minimum and maximum of the range specified as 0.5 to 4 DN/s.
# These limits can be adjusted based on examination of the pixel values
# (e.g., a histogram).
norm = simple_norm(file.data, 'asinh', vmin=0.5, vmax=4)

fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.imshow(file.data, norm=norm, origin='lower')
ax.set_xlabel('X Science Axis (pixels)')
ax.set_ylabel('Y Science Axis (pixels)')
ax.set_title('Roman I-Sim Simulation WFI01')
plt.colorbar(sc, ax=ax)
plt.tight_layout();

We can see a bright, extended sources in the right portion of the image. We isolate and examine that region a little more closely. Based on the image above, let's isolate science Y coordinates 2000 – 2500 and science X coordinates 3500 - 4000.

In [ ]:
# Close the previous figure
plt.close()

# Make new figure zoomed in on 1750 <= X <= 2250 and 500 <= Y <= 1000:
fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.imshow(file.data, norm=norm, origin='lower')
ax.set_xlabel('X Science Axis (pixels)')
ax.set_ylabel('Y Science Axis (pixels)')
ax.set_title('Roman I-Sim Simulation (Data) WFI01')
ax.set_xlim(3500, 4088)
ax.set_ylim(2000, 2588)
plt.colorbar(sc, ax=ax)
plt.tight_layout();

We can see some strange features in the right side of this image. They were also in the previous plot, but were not as evident at the previous level of zoom. These features are not some interesting new science, but are actually instrumental artifacts, and we can see in the data quality (DQ) array that they are marked for our reference in any analysis we may perform. Not all artifacts or DQ flags are necessarily bad for analysis, but here we show all pixels with DQ values > 0 (any quality flags; white) compared to pixels with DQ values equal to 0 (good; black). For more information on how to work with DQ flags, see the [Working with ASDF](../working_with_asdf/working_with_asdf.ipynb) tutorial. The [Detector Performance](https://roman-docs.stsci.edu/roman-instruments-home/wfi-imaging-mode-user-guide/wfi-detectors/detector-performance) article and the articles under it will provide detail on instrumental artifacts and detector performance. Please note that several analyses are still ongoing and RDox will be updated in the future.

In [ ]:
# Close the previous figure
plt.close()

# Display the data quality (DQ) array for the same region.
# Convert the DQ values to boolean True (bad) and False (good) for simple display.
# The binary colormap normally goes from white to black, but we have inverted it
# using binary_r to ease visibility of flagged pixels.
fig, ax = plt.subplots(figsize=(8, 6))
ax.imshow(np.bool(file.dq), origin='lower', cmap='binary_r')
ax.set_xlabel('X Science Axis (pixels)')
ax.set_ylabel('Y Science Axis (pixels)')
ax.set_title('Roman I-Sim Simulation (DQ) WFI01')
ax.set_xlim(3500, 4088)
ax.set_ylim(2000, 2588)
plt.tight_layout();

If we want to add information about the sky coordinates rather than displaying the image in pixels, we can use the `gwcs.wcs.WCS` object in the file metadata. To overlay the coordinate grid, however, we will need to transform the WCS to an `astropy.wcs.WCS` object, which needs a FITS Simple Imaging Polynomial (SIP) representation of the WCS. 

In [ ]:
# Close the previous figure
plt.close()

# Set the matplotlib backend (change this to "%matplotlib widget" for interactive plots)
%matplotlib inline

fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': WCS(file.meta.wcs.to_fits_sip())})
sc = ax.imshow(file.data, norm=norm, origin='lower')
ax.imshow(file.data, norm=norm, origin='lower')
plt.colorbar(sc, ax=ax)
ax.grid(color='white', alpha=0.3)
ax.set_xlabel('Right Ascension (deg)')
ax.set_ylabel('Declination (deg)')
plt.show()

In [ ]:
# Close figures before proceeding
plt.close('all')

We can also use `matplotlib` to examine the image interactively using the `%matplotlib widget` magic command. To do this, try re-running the cell above, but replacing `%matplotlib inline` with `%matplotlib widget`. If you use this command, you will see some basic icons on the left of the image that allow you to pan, zoom, return to the original display, and save the current view to a file. Notice that the WCS gridlines will adjust for different levels of zoom. You can do this for any of the `matplotlib` plots above. For even more interactivity, we next look at Imviz.

## Interactively Examine the Data Using Imviz

### Introduction and Setup

We can also use Imviz, the 2-D image viewer from the Jdaviz package, to visualize and explore the 2-D arrays contained within WFI L2 ASDF files. We highly recommend that users consult the [Imviz documentation](https://jdaviz.readthedocs.io/en/latest/imviz/index.html), which describes many of the features in Imviz in detail. In this tutorial, we will cover the basics to get you started.

The first cell below loads Imviz 

In [ ]:
# Load Imviz
imviz = Imviz()

### Loading Data

Next, we load the image into Imviz. By default, for Roman WFI data, Imviz only loads the data array in the viewer to improve performance. Additional arrays (e.g., the DQ array) may be loaded using the `ext` optional argument. An example demonstrating how to load the data quality array is provided in a commented line in the following cell. For more information on the arrays contained within WFI L2 files, please see the RDox article on WFI [Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products#DataLevelsandProducts-L2ScienceDataSpecifications).

In [ ]:
asdf_dir_uri = 's3://stpubdata/roman/nexus/soc_simulations/tutorial_data/'
fs = s3fs.S3FileSystem(anon=True)

asdf_file_uri = asdf_dir_uri + 'r0003201001001001004_0001_wfi01_f106_cal.asdf'
file = rdm.open(fs.open(asdf_file_uri, 'rb'))
wcs1 = file.meta.wcs

In [ ]:
# loading on Imviz
imviz.load(file, data_label='WFI01_POS1')
# imviz.load_data(file, ext='dq', data_label='WFI01 DQ')

time.sleep(10)

### Programmatically Adjusting Display

Next, we load the image into Imviz. By default, for Roman WFI data, Imviz only loads the data array in the viewer to improve performance. Additional arrays (e.g., the DQ array) may be loaded using the `ext` optional argument. An example demonstrating how to load the data quality array is provided in a commented line in the following cell. For more information on the arrays contained within WFI L2 files, please see the RDox article on WFI [Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products#DataLevelsandProducts-L2ScienceDataSpecifications).

In Jupyter Lab the viewer will appear at the right of the notebook in Jupyter Lab, but if not possible, the tool will use the visualization tool `inline` that will show underneat the cell.

In [ ]:
inviz_display = None
try:
    imviz.show(loc="sidecar", height=1000)
except Exception:
    imviz.show(loc="inline")   # or "popout"
    inviz_display = "inline"
    print(f"Using inline display")

Using `loc=sidecar` creates a split panel to the right. You can control the height, or use a pop out window or an inline viewer. For more information, please see the [display options](https://jdaviz.readthedocs.io/en/latest/display.html) documentation.

**Note:** You may need to expand the Imviz window to see all of the menubar items. You can also minimize the viewer by clicking on the tab on the far-right of your browser that says "imviz." Clicking the "x" symbol next to it will close the tab, but clicking on the word "imviz" will hide the viewer until it is clicked again.

The colormap, stretch, and scale limits may be adjusted interactively in the viewer by clicking the plot options icon (three vertically-stacked sliders to the far right of the teal bar above the image display, hereafter the "tool bar"), and expanding the "Plot Options" section. Note that there are additional options such as the minimum and maximum scale limits under the "More Stretch Options" expander.

If we know the settings we want to apply, we can do so via the Imviz API as follows:

In [ ]:
viewer = imviz.default_viewer
viewer.stretch_options
viewer.stretch = 'arcsinh'
viewer.set_colormap('Viridis')
viewer.cuts = (0.5, 4)
time.sleep(5)

if inviz_display == "inline":
    imviz.show(loc="inline")

**Note:** We have added a few pauses in the notebook cells with, e.g., `time.sleep(5)`, which should cause the cell to pause for several seconds before completing. Some cells include a retry pattern with a longer wait period. This is to ensure that the previous cell has had enough time to complete its tasks before moving on to the next step. These pauses or retry patter may make Imviz feel a little slow, but it is meant to slow you down so you don't try to issue new commands before the previous ones finish executing. You may still want to execute the following cells slowly to prevent errors. **Important:** if you do get an error message, try re-running the cell and waiting a moment.

Now we will display a second image, showing the same field but slightly dithered, and we will link the two images by their WCS. The second image is dithered by ~200 arcseconds compared to the first, so sources should move by ~1000 pixels between the two images.

In [ ]:
asdf_file_uri = asdf_dir_uri + 'r0003201001001001004_0002_wfi01_f106_cal.asdf'
f2 = fs.open(asdf_file_uri, 'rb')
file2 = rdm.open(f2)
wcs2 = file2.meta.wcs
imviz.load(file2, data_label='WFI01_POS2')

In [ ]:
# Adjust wating period for viewer
max_attempts = 20  # Allows up to ~2 minutes total wait
attempt = 0
while attempt < max_attempts:
    try:
        viewer = imviz.default_viewer
        viewer.stretch_options
        viewer.stretch = 'arcsinh'
        viewer.set_colormap('Viridis')
        viewer.cuts = (0.5, 4)  
        # If it succeeds without raising, break out of the loop
        break
    except ValueError as e:
        if 'Viewer is still loading' in str(e):
            attempt += 1
            print(f"Viewer not ready yet (attempt {attempt}/{max_attempts}), waiting...")
            time.sleep(5)  # Or 3-10 seconds per retry
        else:
            raise  # Re-raise if it's a different error
else:
    raise RuntimeError("Viewer failed to load after maximum attempts")

if inviz_display == "inline":
    imviz.show(loc="inline")

Now that the second image has been displayed, we can blink between the two using the "b" button. Make sure that your cursor is placed over the image to make it the active window, then blink between the images. Try identifying a few sources that are common between the two images to see the effect of dithering between exposures.

Similarly to our matplotlib demonstration, let's investigate a region of interest and focus on the extended source in the right portion of the first image. In this case, we know the galaxy has science pixel coordinates of (X, Y) ~ (3700, 2300) pixels in the first image. We can use the WCS from the first image (which we saved as the variable `wcs1`) to transform this to sky coordinates:

In [ ]:
ra, dec = wcs1(3700, 2300)
print(f'RA = {ra} deg, Dec = {dec} deg')

Next, we can center the viewer at this position using the Imviz API as shown below. We can use either the sky coordinates, or alternatively the pixel coordinates (which has been commented out). Note that the centering occurs on the currently displayed image, so if you have the second image actively displayed and use the pixel coordinates, it will center to the incorrect location when using pixel coordinates from the first image. Using sky coordinates is independent of which image is actively displayed.

In [ ]:
# Center the image on given sky coordinates.
sky = SkyCoord(ra=ra, dec=dec, unit=('deg', 'deg'))
viewer.center_on(sky)

# Center the image on given pixel coordinates.
# viewer.center_on((3700, 2300))

time.sleep(5)

if inviz_display == "inline":
    imviz.show(loc="inline")

We can also set the zoom level to better display the region around the extended source. The zoom level settings are such as:

* 1: real-pixel-size.
* 2: zoomed in by a factor of 2.
* 0.5: zoomed out by a factor of 2.
* 'fit' means zoomed to fit the whole image into display.

In this case, we will set the zoom level to 1.2 so we can better see the extended source. Making this plot takes time, so we adjust the wait time period to allow Imviz to catch up before rendering the images.

In [ ]:
# Adjust wating period for viewer
max_attempts = 20  # Allows up to ~2 minutes total wait
attempt = 0
while attempt < max_attempts:
    try:
        viewer.zoom_level = 1.2
        # If it succeeds without raising, break out of the loop
        break
    except ValueError as e:
        if 'Viewer is still loading' in str(e):
            attempt += 1
            print(f"Viewer not ready yet (attempt {attempt}/{max_attempts}), waiting...")
            time.sleep(5)  # Or 3-10 seconds per retry
        else:
            raise  # Re-raise if it's a different error
else:
    raise RuntimeError("Viewer failed to load after maximum attempts")

if inviz_display == "inline":
    imviz.show(loc="inline")

Recall that to blink the images, you need to place the cursor over the viewer and press the "b" key on your keyboard. As you can see, when blinking the images in detector coordinates, the extended source is only visible in one image due to dithering between the exposures. Next, we will link the images by their WCS information so that the sources remain fixed in the display. Linking the images by WCS will reset the center and zoom, so let's reapply that as well.

**Important note:** Roman WFI ASDF files use a Generalized World Coordinate System (GWCS) object in Python to store the WCS transformation. The transformation is only well-defined within a bounding box, and moving outside that bounding box produces unexpected behavior, particularly due to the polynomial terms in the transformation between pixel and sky coordinates. As a result, the current version of Imviz might have some difficulties with setting a position and zooming after linking the images by WCS.

In [ ]:
orientation = imviz.plugins['Orientation']
orientation.align_by = 'WCS'
time.sleep(5)

if inviz_display == "inline":
    imviz.show(loc="inline")

We can still pan and zoom manually to locate our galaxy of interest and use the 'b' button with the cursor on the Imviz window to blink between the two images. Blinking between the two images in any region where they overlap on the sky shows that we have succesfully aligned the images using sky coordinates.

You can also save the current view to a PNG file on your Nexus storage:

In [ ]:
viewer.save('my_image.png')
time.sleep(5)

This should save the file to your current working directory on the Nexus. If you want to download this file, you can do so by right-clicking on the file in the file browser and selecting the "Download" option.

### Overlaying Catalog Data

A common exercise is to mark sources of interest on an image. We can do this with Imviz as well. First, let's read in the catalog used to generate this image using Roman I-Sim:

In [ ]:
# Read in catalog data from S3
cat_uri = asdf_dir_uri + 'full_catalog.ecsv'
tab = Table.read(fs.open(cat_uri, 'rb'), format='ascii.ecsv')

There are a lot of sources in this file, but let's pare them down somewhat for display purposes. In this case, let's filter down to the brightest ($m_{AB}$ < 18) sources in the F106 filter. The flux columns in the table are in units of maggies, which may be converted to AB magnitudes as $m_{AB} = -2.5\log_{10}(f)$ where $m_{AB}$ is the magnitude in AB mags and $f$ is the flux in maggies.

Let's also filter the catalog for sources that lie in the overlap region between the two images. We can do this by transforming our sky positions to pixel positions with the WCS objects from each image and creating a mask by combining multiple conditions on the pixel positions (i.e., that they must lie within the bounds 0 <= X <= 4088 and 0 <= Y <= 4088).

In [ ]:
# Filter objects by brightness
bright = np.where(-2.5 * np.log10(tab['F106']) < 18)

# Make SkyCoord objects
coords = SkyCoord(ra = tab['ra'][bright].value, dec = tab['dec'][bright].value, unit = 'deg')

# Filter the SkyCoord objects on the WCS objects for each image to find only things in the overlap region.
x1, y1 = wcs1.invert(coords)
x2, y2 = wcs2.invert(coords)
mask = (x1 > 0) & (x1 < 4088) & (y1 > 0) & (y1 < 4088) & (x2 > 0) & (x2 < 4088) & (y2 > 0) & (y2 < 4088)

final_coords = Table({'coord': coords[mask]})
print(f'Number of sources in overlap region: {len(final_coords)}')

Now let's set up the markers and add them to the viewer:

In [ ]:
# Set up viewer marker parameters
viewer.marker = {'color': 'white', 'alpha': 0.8, 'markersize': 120, 'fill': False}

# Overlay markers
viewer.add_markers(final_coords, use_skycoord=True, marker_name='My_Markers')
time.sleep(5)

if inviz_display == "inline":
    imviz.show(loc="inline")

We can also remove them if we don't want them any longer, or if we want to replace them with other markers:

In [ ]:
# Remove only My_Markers
viewer.remove_markers(marker_name='My_Markers')

# Remove ALL markers
# viewer.reset_markers()

if inviz_display == "inline":
    imviz.show(loc="inline")

For more advanced use cases such as interactive aperture photometry or analysis of line profiles, please consult the [Imviz documentation](https://jdaviz.readthedocs.io/en/latest/imviz/index.html).

***

## Additional Resources

- [Imviz Documentation](https://jdaviz.readthedocs.io/en/latest/imviz/index.html)
- [Additional JDaviz Notebooks](https://github.com/spacetelescope/jdaviz/tree/main/notebooks)
- [RDox WFI Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products#DataLevelsandProducts-L2ScienceDataSpecifications)

## About this Notebook
**Author:** Tyler Desjardins, Brett Morris, R. Diaz  
**Updated On:** 2025-12-13

<table width="100%" style="border:none; border-collapse:collapse;">
  <tr style="border:none;">
    <td style="border:none; width:180px; white-space:nowrap;">
       <a href="#top" style="text-decoration:none; color:#0066cc;">↑ Top of page</a> 
    </td>
    <td style="border:none; text-align:center;">
       <img src="../../roman_logo.png" width="50">
    </td>
    <td style="border:none; text-align:right;">
       <img src="../../stsci_logo2.png" width="90">
    </td>
  </tr>
</table>